# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [2]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = 'qPiYCsYNsQK0FKgX0j5zJo06nuMKZmvNpnx9Rpqe'  # Please replace by your own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [3]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [6]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'shout',
        'filter': 'duration:[0 TO 2]',
        #'filter': 'ac_reverb: True',
        'num_results': 10, 
    },
    {
        'query': 'kick_drum',
        'filter': 'duration:[0 TO 2]',
        'num_results': 10, 
    },
    {
        'query': 'snare_drum',
        'filter': 'duration:[0 TO 2]',
        'num_results': 10, 
    },
    {
        'query': 'dj_scratch',
        'filter': 'duration:[0 TO 2]',
        #'filter': 'ac_brightness:[75 TO 100]',
        'num_results': 10, 
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 40 entries! dataframe.csv


,freesound_id,license,name,path,tags,username
0,469308,http://creativecommons.org/publicdomain/zero/1.0/,Voice Shout #06,files/469308_2927958-hq.ogg,"[male, field-recording, unprocessed, vocal, ra...",bolkmar
1,100759,http://creativecommons.org/licenses/by-nc/3.0/,01677 fight shout.wav,files/100759_321967-hq.ogg,"[battle, fight, game, hit, karate, kung-fu, mo...",Robinhood76
2,220662,http://creativecommons.org/licenses/by/3.0/,shout.wav,files/220662_3824909-hq.ogg,"[666moviescream, upf, frighten, shout, movie, ...",marionagm90
3,503020,http://creativecommons.org/publicdomain/zero/1.0/,dog shout 2,files/503020_3953729-hq.ogg,"[growl, shout, dog]",annstory4520
4,436782,http://creativecommons.org/publicdomain/zero/1.0/,scary shout,files/436782_4347097-hq.ogg,"[rawr, scary, phantom, monster, roar, spooky, ...",Alivvie
5,85897,http://creativecommons.org/licenses/by/3.0/,HUH! 01.wav,files/85897_14771-hq.ogg,"[guttural, huh, male, percussive, shout]",sandyrb
6,509882,http://creativecommons.org/publicdomain/zero/1.0/,female_shout_01.wav,files/509882_9238038-hq.ogg,"[shouting, shout, female]",slamaxu
7,509885,http://creativecommons.org/publicdomain/zero/1.0/,male_shout_01.wav,files/509885_9238038-hq.ogg,"[shouting, shout, male]",slamaxu
8,220820,http://creativecommons.org/publicdomain/zero/1.0/,Awful shout man.wav,files/220820_3559284-hq.ogg,"[666moviescreams, bilis, shout, vomit, awful, ...",EricSueiro
9,551019,http://creativecommons.org/licenses/by/3.0/,shouting in.wav,files/551019_7495013-hq.ogg,"[body, sound, male, inside, vocal, breath, sho...",bogenseeberg
